In [17]:
## Задание 1
### Будем парсить страницу со свежими новостям на habr.com/ru/all/.
### Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
### KEYWORDS = ['python', 'парсинг']
### Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
### В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [18]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

KEYWORDS = set(['python', 'парсинг', 'факторизация', 'месяц'])

req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

posts = soup.find_all('article', class_='post')

In [19]:
def get_data_from_post(post, check, df):
    text = set(check.lower().split())
    if bool(text.intersection(KEYWORDS)):
        date_element = post.find('span', class_='post__time')
        title_element = post.find('a', class_='post__title_link')
        df = df.append({'date': date_element.text, 'title': title_element.text,
                        'link': title_element.attrs.get('href')}, ignore_index=True)
        return True, df
    return False, None

In [20]:
df = pd.DataFrame(columns=['date', 'title', 'link'])
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs: # поиск по тегам
        rc, res = get_data_from_post(post, hub.text, df)
        if rc:
            df = res
            continue
    title = post.find('a', class_='post__title_link') # поиск по имени статьи
    rc, res = get_data_from_post(post, title.text, df)
    if rc:
        df = res
        continue
    text = post.find('div', class_='post__text') # поиск по превью статьи
    rc, res = get_data_from_post(post, text.text, df)
    if rc:
        df = res
        continue
df

,date,title,link
0,сегодня в 00:04,Маркировка остатков немаркированных сигарет. И...,https://habr.com/ru/post/525652/
1,вчера в 18:26,Настройка синхронизации google calendar с web ...,https://habr.com/ru/post/525680/
2,вчера в 18:11,Факторизация чисел и сумма неизвестных делител...,https://habr.com/ru/post/524798/


In [21]:
## Задание 2
### Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
### EMAIL = [xxx@x.ru, yyy@y.com]
### В итоге должен формироваться датафрейм со столбцами: < почта > - < дата утечки > - < источник утечки > - < описание утечки >
### Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [22]:
import json

EMAIL = ['lol@rambler.ru', 'lol@mail.ru']

url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data' # скрытый метод проверки на утечки
params = {
    'emailAddresses': EMAIL
}
headers = {
    'Vaar-Version': '0', # необходимые заголовки для доступа к методу проверки почты на утечку
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Content-Type': 'application/json;charset=UTF-8',
}

res = requests.post(url, headers=headers,
                    data=json.dumps(params, separators=(',', ':'))).json() # формат принимаемого JSON-объекта в методе проверки почты на утечку не поддерживает пробелы в разделителях, используемых по-умолчанию
df = pd.DataFrame(columns=['email', 'date', 'origin', 'description'])
for email in res['summary']:
    for breach in res['summary'][email]['breaches']:
        df = df.append({'email': email,
                        'date': res['breaches'][str(breach)]['publishDate'],
                        'origin': res['breaches'][str(breach)]['site'],
                        'description': res['breaches'][str(breach)]['description']},
                       ignore_index=True)
df

,email,date,origin,description
0,lol@rambler.ru,2019-07-25T00:00:00Z,worldofwar.net,"At an unconfirmed date, World of Warcraft fand..."
1,lol@rambler.ru,2020-07-23T00:00:00Z,wattpad.com,"In June 2020, the online writing community Wat..."
2,lol@rambler.ru,2018-08-24T00:00:00Z,forums.funny-games.biz,"In April 2018, Funny Games' Forums database wa..."
3,lol@rambler.ru,2019-10-17T00:00:00Z,zynga.com,"In September 2019, the game developer Zynga wa..."
4,lol@rambler.ru,2020-04-09T00:00:00Z,vkmix.com,"At an unconfirmed date, the Russian social med..."
5,lol@mail.ru,2019-03-20T00:00:00Z,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
6,lol@mail.ru,2019-03-20T00:00:00Z,roll20.net,"In January 2019, Roll20 allegedly breached. Th..."
7,lol@mail.ru,2020-08-27T00:00:00Z,plibber.ru,"At an unconfirmed date, the Russian advertisin..."
8,lol@mail.ru,2019-07-25T00:00:00Z,worldofwar.net,"At an unconfirmed date, World of Warcraft fand..."
9,lol@mail.ru,2020-05-28T00:00:00Z,stalker.so,"In January 2020, the Russian multiplayer video..."
